In [6]:
library(data.table)

In [9]:
trans_dir = "/directflow/SCCGGroupShare/projects/anncuo/OneK1K/saige_eqtl/from_wei/trans_CD4_NC_tophits_multianno/"
# trans_dir = "/directflow/SCCGGroupShare/projects/anncuo/OneK1K/saige_eqtl/from_wei/trans_B_IN_tophits_multianno/"
# trans_dir = "/directflow/SCCGGroupShare/projects/anncuo/OneK1K/saige_eqtl/from_wei/trans_Plasma_tophits_multianno/"

trans_files = list.files(trans_dir, pattern = "gw_allchr.txt")
length(trans_files)
head(trans_files,2)

[1] 10527

[1] "A1BG_B_IN_count_gw_allchr.txt_result.tophits.annovar.txt.hg19_multianno.txt"
[2] "AAAS_B_IN_count_gw_allchr.txt_result.tophits.annovar.txt.hg19_multianno.txt"

In [18]:
# Manual pruning of "independent" SNPs based on distance

dist = 500000
for (myfile in trans_files){
    df = as.data.frame(fread(paste0(trans_dir,myfile)))
    df = unique(df)
    colnames(df)[(ncol(df)-2):ncol(df)] <- c("p.value","neg.log10.p.value","cis.trans")
    # filter by p-value
    df = df[df$p.value < 5e-8,]
    if (nrow(df)==0){next}
    df_list = list()
    to_include_snps = c()
    for (chrom in unique(df$Chr)){
        snps = df[df$Chr==chrom,"Start"]
        if (length(snps)==1){chrom_snps = snps}
        else {
            chrom_snps = snps[1]
            for (i in 2:length(snps)){
                snp = snps[i]
                mydist = snp-chrom_snps[length(chrom_snps)]
                if (mydist>dist){chrom_snps = c(chrom_snps, snp)}
            }
        }
        to_include_snps = c(to_include_snps,chrom_snps)
        df_list[[chrom]] = df[df$Chr==chrom & df$Start %in% to_include_snps, ]
    }
    df_new = rbindlist(df_list)
    out_file = paste0(trans_dir,"merged_p5e_8/",myfile,".merged.txt")
    fwrite(df_new, out_file)
}